### Correlation and Relationships

Aktuell (im letzten NOtebook) stehen die Werte untereinander (eine Zeile pro Measure). Um zu testen, ob "Sicherheit" mit "Lebenszufriedenheit" korreliert, müssen diese als Spalten nebeneinander stehen. Wir müssen den Datensatz also umformen ("pivoten").

-> Bevor wir eine Zahl berechnen, müssen wir die Daten sehen. Ist der Zusammenhang linear? Gibt es "Wolken"?

### 1. Setup & Daten Laden

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path

In [2]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
current_dir = Path.cwd()
data_path = current_dir.parent / 'data' / 'oecd_snapshot_latest.csv'

#try:
#    df = pd.read_csv(data_path)
#    print("Daten erfolgreich geladen.")
#except FileNotFoundError:
#    print("FEHLER: Datei nicht gefunden. Pfad prüfen!")

try:
    df = pd.read_csv(data_path)

    df = df.loc[:, ~df.columns.duplicated()]
    
    print("Daten erfolgreich geladen.")
    print(f"Spalten im Datensatz: {df.columns.tolist()}")
    
except FileNotFoundError:
    print(f"FEHLER: Datei nicht gefunden unter {data_path}")

Daten erfolgreich geladen.
Spalten im Datensatz: ['reference_area', 'measure', 'unit_of_measure', 'age', 'sex', 'education_level', 'domain', 'year', 'value']


### 2. Daten Transformation

In [16]:
print("--- Kategorien-Check ---")
print(f"Sex: {df['sex'].unique()}")
print(f"Age: {df['age'].unique()}")
if 'education_level' in df.columns:
    print(f"Education: {df['education_level'].unique()}")

--- Kategorien-Check ---
Sex: ['Total' 'Male' 'Female']
Age: ['Total' 'Young' 'Old' 'Middle-aged']
Education: ['Secondary education' 'Primary education' 'Tertiary education' 'Total']


In [17]:
target_sex = 'Total' if 'Total' in df['sex'].unique() else '_T'
target_age = 'Total' if 'Total' in df['age'].unique() else '_T'

In [18]:
mask = (df['sex'] == target_sex) & (df['age'] == target_age)

In [19]:
# Wenn wir nicht auf Education filtern, haben wir Duplikate!
if 'education_level' in df.columns:
    # Suche nach 'Total', '_T' oder ähnlichem
    target_edu = 'Total' if 'Total' in df['education_level'].unique() else \
                 ('_T' if '_T' in df['education_level'].unique() else None)
    
    if target_edu:
        print(f"Filtere zusätzlich auf Education = '{target_edu}'")
        mask = mask & (df['education_level'] == target_edu)
    else:
        print("Warnung: Kein 'Total' für Education gefunden. Nehme alle (Gefahr von Duplikaten).")

df_filtered = df[mask].copy()

Filtere zusätzlich auf Education = 'Total'


In [20]:
# Gibt es für (Land, Measure) immer noch mehr als einen Wert?
dupes = df_filtered.duplicated(subset=['reference_area', 'measure'], keep=False)
if dupes.any():
    print(f"\nACHTUNG: Immer noch {dupes.sum()} Duplikate gefunden!")
    print(df_filtered[dupes][['reference_area', 'measure', 'education_level', 'value']].head())
    
    # Notfall-Lösung: Wir nehmen den Mittelwert der verbleibenden Duplikate
    print("-> Aggregiere Duplikate mit Mittelwert (Mean)...")
    df_pivot = df_filtered.pivot_table(index='reference_area', columns='measure', values='value', aggfunc='mean')
else:
    # Alles sauber -> normaler Pivot
    df_pivot = df_filtered.pivot(index='reference_area', columns='measure', values='value')

print(f"\nDatensatz erfolgreich transformiert. Anzahl Länder: {len(df_pivot)}")


Datensatz erfolgreich transformiert. Anzahl Länder: 47


### 3. VARIABLEN SELEKTION

In [22]:
# Unsere Zielvariable aus Notebook 2
target_var = "Feeling safe at night"

# Verfügbare Indikatoren prüfen
print("\nVerfügbare Indikatoren:")
print(df_pivot.columns.tolist()[:10])


Verfügbare Indikatoren:
['Access to green space', 'Adult literacy skills', 'Adult numeracy skills', 'Adults with low literacy skills', 'Adults with low numeracy skills', 'Average annual gross earnings', 'Deaths from suicide, alcohol, drugs', 'Difficulty making ends meet', 'Employment rate', 'Equivalised liquid financial assets below three months of the annual national relative income poverty line']


**Hypothesen zur Variablenwahl**: Wir untersuchen den Zusammenhang zwischen dem subjektiven Sicherheitsgefühl (Feeling safe at night) und folgenden Indikatoren:

**Ökonomische Stabilität** (Average annual gross earnings, Employment rate):

Hypothese: Länder mit höherem Einkommen und Beschäftigungsgrad haben tendenziell niedrigere Kriminalitätsraten und bessere Infrastruktur (Beleuchtung, Polizei), was das Sicherheitsgefühl erhöht.

**Soziale Notlage** (Difficulty making ends meet):

Hypothese: Finanzielle Sorgen korrelieren oft mit sozialer Instabilität. Wir erwarten eine negative Korrelation (je schwerer es ist, über die Runden zu kommen, desto unsicherer fühlen sich die Menschen).

**Indikatoren für soziale Verzweiflung** (Deaths from suicide, alcohol, drugs):

Hypothese: Diese sogenannten "Deaths of Despair" sind ein Indikator für tiefgreifende gesellschaftliche Probleme. Ein Land mit hohen Werten hier könnte auch als "sozial unsicherer" wahrgenommen werden (Negative Korrelation).

**Umweltfaktoren** (Access to green space):

Hypothese: Zugang zu Grünflächen steht oft für eine höhere Lebensqualität und städtebauliche Ordnung ("Broken Windows Theory" - gepflegte Umgebungen wirken sicherer). Wir erwarten eine positive Korrelation.

In [23]:
# Wir wählen Indikatoren aus verschiedenen Bereichen (Ökonomie, Gesundheit, Umwelt)
potential_drivers = [
    "Average annual gross earnings",       # Hypothese: Reichtum -> Sicherheit
    "Employment rate",                     # Hypothese: Jobs -> Stabilität -> Sicherheit
    "Deaths from suicide, alcohol, drugs", # Hypothese: "Deaths of Despair" -> Soziale Unsicherheit
    "Access to green space",               # Hypothese: Gepflegte Umgebung -> Sicherheit (Broken Windows Theory)
    "Difficulty making ends meet"          # Hypothese: Armut -> Unsicherheit
]

#DataFrame filtern auf Spalten, die wirklich da sind
existing_cols = [c for c in [target_var] + potential_drivers if c in df_pivot.columns]
df_corr = df_pivot[existing_cols].copy()

# Wir löschen Länder, die in diesen Spalten NaNs haben (Listwise Deletion) -> Korrelation braucht vollständige Paare
df_corr = df_corr.dropna()

print(f"\nAnalyse-Tabelle erstellt mit {df_corr.shape[0]} Ländern.")
df_corr.head()


Analyse-Tabelle erstellt mit 26 Ländern.


measure,Feeling safe at night,Average annual gross earnings,Employment rate,"Deaths from suicide, alcohol, drugs",Access to green space,Difficulty making ends meet
reference_area,,,,,,
Austria,83.0,71166.761,78.197,20.523503,77.252527,13.1
Belgium,71.0,73205.848,75.825,21.705963,57.652810,17.2
Czechia,75.5,37365.641,85.498,20.053676,86.247004,14.2
Denmark,87.5,69524.927,81.527,15.434958,59.497138,12.8
Estonia,84.0,37403.843,83.415,42.315857,72.850231,9.9


### 4. Visuelle Prüfung

x_val = "Life satisfaction"
y_val = target_var